In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
import sys

#Set device
USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    device = torch.device("cuda")
    cuda = True
else:
    device = torch.device("cpu")
    cuda = False
    
print("Device =", device)
gpus = [0]

Device = cuda


In [2]:
def time_elapsed(start_time):
    elapsed = time.time() - start_time
    hours = int(elapsed/3600)
    minutes = int(int(elapsed/60)%60)
    seconds = int(elapsed%60)
    
    return hours, minutes, seconds

In [3]:
#Load SMILES data as integer labels and as one-hot encoding
data = np.load("ohesmiles.npz")
data = data["arr_0"]

intdata = np.load("intsmiles.npz")
intdata = intdata["arr_0"]

data = torch.from_numpy(data).view(np.shape(data)[0], 1, np.shape(data)[1])
intdata = torch.from_numpy(intdata)

print("Dataset size: " + str(data.size()))
print("Integer dataset size: " + str(intdata.size()))

Dataset size: torch.Size([25246858, 1, 53])
Integer dataset size: torch.Size([25246858])


In [4]:
#Get input tensor
def inptarg(i, batch_size):
    while True:
        try:
            
            #Define input and target tensor sizes
            inp = torch.Tensor((seq_length - 1) * batch_size, 1, np.shape(data)[2])
            target = torch.Tensor((seq_length - 1) * batch_size)
    
            #SMILES molecules in batch 
            inputs = data[int((i * seq_length)) : int((i * seq_length) + (seq_length * batch_size)), :, :]
            targets = intdata[int((i * seq_length)) : int((i * seq_length) + (seq_length * batch_size))]

            #Index counters for input, target
            r = 0
            s = 0 
            for p in range(seq_length * batch_size - 1):
        
                if (p % seq_length != (seq_length - 1)):
                    #Input data (does not include last character in SMILES)
                    inp[r, :, :] = inputs[p, :, :]
                    r += 1
            
                if (p % seq_length != 0):
                    #Target data (does not include first character in SMILES)
                    target[s] = targets[p]
                    s += 1
                    
            return inp, target
        except:
            continue
        break

In [5]:
#Define model
class Model(nn.Module):
    
    #Define model parameters
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(Model, self).__init__()
        
        #Model parameters
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        
        #Model layers
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.linear = nn.Linear(hidden_size, input_size)
        
    #Define initial hidden and cell states
    def init_states(self, num_layers, hidden_size):
        hidden = [Variable(torch.zeros(num_layers, 1, hidden_size)),
                  Variable(torch.zeros(num_layers, 1, hidden_size))]
        
        return hidden
    
    #Define forward propagation
    def forward(self, inp, hidden):
        #LSTM
        output, hidden = self.lstm(inp, hidden)

        #Linear Layer
        output = self.linear(output)
        
        return output, hidden
    

In [6]:
#Define training
def train(epochs):
    #Set start time
    start_time = time.time()
    
    #Iterate over desired number of epochs 
    for e in range(epochs):
        
        #Iterate set of seq_length characters
        for i in range((np.shape(data)[0] - seq_length + 1) / batch_size):
            
            #Initialize hidden and cell states
            hidden = model.init_states(num_layers, hidden_size)
            
            #Run on GPU if available
            if cuda:
                hidden = (hidden[0].cuda(), hidden[1].cuda())
        
            #Set initial gradients
            model.zero_grad()
    
            #Set initial loss
            loss = 0 
            
            #Get input and target
            input_data, target_data = inptarg(i, batch_size)
            input_data = input_data.float()
            target_data = target_data.long()
            
            #Run on GPU if available
            if cuda:
                input_data = input_data.cuda()
                target_data = target_data.cuda()
                
            #Run model, calculate loss
            output, hidden = model(input_data, hidden)
            loss += criterion(output.squeeze(), target_data.squeeze())
                
            #Backpropagate loss
            loss.backward()
            optimizer.step()
            
            #Update list of losses
            if (e == i == 0):
                losses[0] = loss.data.item() / seq_length   
            losses.append(loss.data.item() / seq_length)
            
            if (i % 100 == 0):
                torch.save(model.state_dict(), "jan15LSTM1024-1.pth")
            
            #Print training info
            hours, minutes, seconds = time_elapsed(start_time)
            print("Loss: {:0.6f}".format(loss.data.item() / seq_length) + " | ΔLossTotal: {:+0.4f}".format(losses[-1] - losses[1]) + " | Epoch: {0:02d}".format(e + 1) + " | Iteration: {0:04d}".format(i + 1) + " | Time elapsed: {0:02d}".format(hours) + "h {0:02d}".format(minutes) + " m {0:02d}".format(seconds) + " s")

In [7]:
#Initialize model parameters
input_size = np.shape(data)[2]
hidden_size = 1024
num_layers = 3
dropout = .2
learning_rate = 0.001
epochs = 1
seq_length = 75
batch_size = 64

#List of losses
losses = [0]

In [8]:
#Call model, set optimizer and loss function
model = Model(input_size, hidden_size, num_layers, dropout)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.CrossEntropyLoss()

#Run on GPU if available
if cuda:
    model.cuda()
    criterion.cuda()  

In [9]:
#Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters in network: " + str(total_params))

Total number of parameters in network: 21267509


In [ ]:
#Train
train(epochs)

Loss: 0.052902 | ΔLossTotal: +0.0000 | Epoch: 01 | Iteration: 0001 | Time elapsed: 00h 00 m 15 s
Loss: 0.051683 | ΔLossTotal: -0.0012 | Epoch: 01 | Iteration: 0002 | Time elapsed: 00h 00 m 30 s
Loss: 0.044397 | ΔLossTotal: -0.0085 | Epoch: 01 | Iteration: 0003 | Time elapsed: 00h 00 m 45 s
Loss: 0.046325 | ΔLossTotal: -0.0066 | Epoch: 01 | Iteration: 0004 | Time elapsed: 00h 01 m 00 s
Loss: 0.039233 | ΔLossTotal: -0.0137 | Epoch: 01 | Iteration: 0005 | Time elapsed: 00h 01 m 15 s
Loss: 0.038246 | ΔLossTotal: -0.0147 | Epoch: 01 | Iteration: 0006 | Time elapsed: 00h 01 m 30 s
Loss: 0.038960 | ΔLossTotal: -0.0139 | Epoch: 01 | Iteration: 0007 | Time elapsed: 00h 01 m 45 s
Loss: 0.038842 | ΔLossTotal: -0.0141 | Epoch: 01 | Iteration: 0008 | Time elapsed: 00h 02 m 00 s
Loss: 0.038059 | ΔLossTotal: -0.0148 | Epoch: 01 | Iteration: 0009 | Time elapsed: 00h 02 m 15 s
Loss: 0.037060 | ΔLossTotal: -0.0158 | Epoch: 01 | Iteration: 0010 | Time elapsed: 00h 02 m 30 s
Loss: 0.036332 | ΔLossTotal: -

Loss: 0.034386 | ΔLossTotal: -0.0185 | Epoch: 01 | Iteration: 0086 | Time elapsed: 00h 21 m 44 s
Loss: 0.034163 | ΔLossTotal: -0.0187 | Epoch: 01 | Iteration: 0087 | Time elapsed: 00h 21 m 59 s
Loss: 0.033960 | ΔLossTotal: -0.0189 | Epoch: 01 | Iteration: 0088 | Time elapsed: 00h 22 m 14 s
Loss: 0.033782 | ΔLossTotal: -0.0191 | Epoch: 01 | Iteration: 0089 | Time elapsed: 00h 22 m 29 s
Loss: 0.033587 | ΔLossTotal: -0.0193 | Epoch: 01 | Iteration: 0090 | Time elapsed: 00h 22 m 44 s
Loss: 0.034470 | ΔLossTotal: -0.0184 | Epoch: 01 | Iteration: 0091 | Time elapsed: 00h 22 m 59 s
Loss: 0.035134 | ΔLossTotal: -0.0178 | Epoch: 01 | Iteration: 0092 | Time elapsed: 00h 23 m 15 s
Loss: 0.033849 | ΔLossTotal: -0.0191 | Epoch: 01 | Iteration: 0093 | Time elapsed: 00h 23 m 30 s
Loss: 0.033678 | ΔLossTotal: -0.0192 | Epoch: 01 | Iteration: 0094 | Time elapsed: 00h 23 m 45 s
Loss: 0.034054 | ΔLossTotal: -0.0188 | Epoch: 01 | Iteration: 0095 | Time elapsed: 00h 24 m 00 s
Loss: 0.033759 | ΔLossTotal: -

Loss: 0.022215 | ΔLossTotal: -0.0307 | Epoch: 01 | Iteration: 0171 | Time elapsed: 00h 43 m 13 s
Loss: 0.022135 | ΔLossTotal: -0.0308 | Epoch: 01 | Iteration: 0172 | Time elapsed: 00h 43 m 28 s
Loss: 0.022355 | ΔLossTotal: -0.0305 | Epoch: 01 | Iteration: 0173 | Time elapsed: 00h 43 m 43 s
Loss: 0.022768 | ΔLossTotal: -0.0301 | Epoch: 01 | Iteration: 0174 | Time elapsed: 00h 43 m 59 s
Loss: 0.022248 | ΔLossTotal: -0.0307 | Epoch: 01 | Iteration: 0175 | Time elapsed: 00h 44 m 14 s
Loss: 0.022184 | ΔLossTotal: -0.0307 | Epoch: 01 | Iteration: 0176 | Time elapsed: 00h 44 m 29 s
Loss: 0.022033 | ΔLossTotal: -0.0309 | Epoch: 01 | Iteration: 0177 | Time elapsed: 00h 44 m 44 s
Loss: 0.022150 | ΔLossTotal: -0.0308 | Epoch: 01 | Iteration: 0178 | Time elapsed: 00h 44 m 59 s
Loss: 0.021953 | ΔLossTotal: -0.0309 | Epoch: 01 | Iteration: 0179 | Time elapsed: 00h 45 m 15 s
Loss: 0.021946 | ΔLossTotal: -0.0310 | Epoch: 01 | Iteration: 0180 | Time elapsed: 00h 45 m 30 s
Loss: 0.021817 | ΔLossTotal: -

Loss: 0.015467 | ΔLossTotal: -0.0374 | Epoch: 01 | Iteration: 0256 | Time elapsed: 01h 04 m 44 s
Loss: 0.015534 | ΔLossTotal: -0.0374 | Epoch: 01 | Iteration: 0257 | Time elapsed: 01h 04 m 59 s
Loss: 0.015551 | ΔLossTotal: -0.0374 | Epoch: 01 | Iteration: 0258 | Time elapsed: 01h 05 m 14 s
Loss: 0.015934 | ΔLossTotal: -0.0370 | Epoch: 01 | Iteration: 0259 | Time elapsed: 01h 05 m 30 s
Loss: 0.015478 | ΔLossTotal: -0.0374 | Epoch: 01 | Iteration: 0260 | Time elapsed: 01h 05 m 45 s
Loss: 0.015481 | ΔLossTotal: -0.0374 | Epoch: 01 | Iteration: 0261 | Time elapsed: 01h 06 m 00 s
Loss: 0.015254 | ΔLossTotal: -0.0376 | Epoch: 01 | Iteration: 0262 | Time elapsed: 01h 06 m 15 s
Loss: 0.015174 | ΔLossTotal: -0.0377 | Epoch: 01 | Iteration: 0263 | Time elapsed: 01h 06 m 30 s
Loss: 0.015100 | ΔLossTotal: -0.0378 | Epoch: 01 | Iteration: 0264 | Time elapsed: 01h 06 m 46 s
Loss: 0.014978 | ΔLossTotal: -0.0379 | Epoch: 01 | Iteration: 0265 | Time elapsed: 01h 07 m 01 s
Loss: 0.014867 | ΔLossTotal: -

Loss: 0.006450 | ΔLossTotal: -0.0465 | Epoch: 01 | Iteration: 0341 | Time elapsed: 01h 26 m 14 s
Loss: 0.006331 | ΔLossTotal: -0.0466 | Epoch: 01 | Iteration: 0342 | Time elapsed: 01h 26 m 30 s
Loss: 0.006240 | ΔLossTotal: -0.0467 | Epoch: 01 | Iteration: 0343 | Time elapsed: 01h 26 m 45 s
Loss: 0.006244 | ΔLossTotal: -0.0467 | Epoch: 01 | Iteration: 0344 | Time elapsed: 01h 27 m 00 s
Loss: 0.006105 | ΔLossTotal: -0.0468 | Epoch: 01 | Iteration: 0345 | Time elapsed: 01h 27 m 15 s
Loss: 0.005986 | ΔLossTotal: -0.0469 | Epoch: 01 | Iteration: 0346 | Time elapsed: 01h 27 m 30 s
Loss: 0.005848 | ΔLossTotal: -0.0471 | Epoch: 01 | Iteration: 0347 | Time elapsed: 01h 27 m 45 s
Loss: 0.005630 | ΔLossTotal: -0.0473 | Epoch: 01 | Iteration: 0348 | Time elapsed: 01h 28 m 01 s
Loss: 0.005621 | ΔLossTotal: -0.0473 | Epoch: 01 | Iteration: 0349 | Time elapsed: 01h 28 m 16 s
Loss: 0.005744 | ΔLossTotal: -0.0472 | Epoch: 01 | Iteration: 0350 | Time elapsed: 01h 28 m 31 s
Loss: 0.005930 | ΔLossTotal: -

...

Loss: 0.001327 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21623 | Time elapsed: 98h 28 m 31 s
Loss: 0.001282 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21624 | Time elapsed: 98h 28 m 47 s
Loss: 0.001178 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 21625 | Time elapsed: 98h 29 m 02 s
Loss: 0.001229 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 21626 | Time elapsed: 98h 29 m 17 s
Loss: 0.001280 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21627 | Time elapsed: 98h 29 m 32 s
Loss: 0.001294 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21628 | Time elapsed: 98h 29 m 47 s
Loss: 0.001287 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21629 | Time elapsed: 98h 30 m 03 s
Loss: 0.001300 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21630 | Time elapsed: 98h 30 m 18 s
Loss: 0.001220 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 21631 | Time elapsed: 98h 30 m 33 s
Loss: 0.001213 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 21632 | Time elapsed: 98h 30 m 48 s
Loss: 0.001423 | ΔLo

Loss: 0.001451 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21707 | Time elapsed: 98h 49 m 44 s
Loss: 0.001432 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21708 | Time elapsed: 98h 49 m 59 s
Loss: 0.001412 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21709 | Time elapsed: 98h 50 m 15 s
Loss: 0.001435 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21710 | Time elapsed: 98h 50 m 30 s
Loss: 0.001403 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21711 | Time elapsed: 98h 50 m 45 s
Loss: 0.001415 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21712 | Time elapsed: 98h 51 m 00 s
Loss: 0.001438 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21713 | Time elapsed: 98h 51 m 15 s
Loss: 0.001681 | ΔLossTotal: -0.0512 | Epoch: 01 | Iteration: 21714 | Time elapsed: 98h 51 m 30 s
Loss: 0.001508 | ΔLossTotal: -0.0514 | Epoch: 01 | Iteration: 21715 | Time elapsed: 98h 51 m 45 s
Loss: 0.001480 | ΔLossTotal: -0.0514 | Epoch: 01 | Iteration: 21716 | Time elapsed: 98h 52 m 01 s
Loss: 0.001341 | ΔLo

Loss: 0.001088 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 21791 | Time elapsed: 99h 55 m 52 s
Loss: 0.001113 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 21792 | Time elapsed: 99h 56 m 41 s
Loss: 0.001237 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 21793 | Time elapsed: 99h 57 m 31 s
Loss: 0.001287 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21794 | Time elapsed: 99h 58 m 20 s
Loss: 0.001201 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 21795 | Time elapsed: 99h 59 m 10 s
Loss: 0.001270 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21796 | Time elapsed: 100h 00 m 00 s
Loss: 0.001263 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21797 | Time elapsed: 100h 00 m 49 s
Loss: 0.001221 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 21798 | Time elapsed: 100h 01 m 39 s
Loss: 0.001270 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21799 | Time elapsed: 100h 02 m 29 s
Loss: 0.001410 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21800 | Time elapsed: 100h 03 m 18 s
Loss: 0.001474 

Loss: 0.001205 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 21874 | Time elapsed: 101h 04 m 43 s
Loss: 0.001401 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21875 | Time elapsed: 101h 05 m 34 s
Loss: 0.001340 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21876 | Time elapsed: 101h 06 m 26 s
Loss: 0.001301 | ΔLossTotal: -0.0516 | Epoch: 01 | Iteration: 21877 | Time elapsed: 101h 07 m 17 s
Loss: 0.001385 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21878 | Time elapsed: 101h 08 m 08 s
Loss: 0.001142 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 21879 | Time elapsed: 101h 08 m 59 s
Loss: 0.001104 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 21880 | Time elapsed: 101h 09 m 50 s
Loss: 0.001141 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 21881 | Time elapsed: 101h 10 m 41 s
Loss: 0.001181 | ΔLossTotal: -0.0517 | Epoch: 01 | Iteration: 21882 | Time elapsed: 101h 11 m 32 s
Loss: 0.001444 | ΔLossTotal: -0.0515 | Epoch: 01 | Iteration: 21883 | Time elapsed: 101h 12 m 21 s
Loss: 0.00

Loss: 0.001081 | ΔLossTotal: -0.0518 | Epoch: 01 | Iteration: 21957 | Time elapsed: 102h 14 m 08 s


In [26]:
torch.save(model.state_dict(), "jan15LSTM1024-1.pth")